<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=eccb4fa778f32c4365a3551c697c98ce82f6aeca3d05b781bde091439080103d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-15 14:22:46
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: 6.24 K (0.04%)
Current PnL: -17.61 L (-11.85%)
CY Booked + Current PnL: -6.58 L (-4.43%)
-------------------
Total profit:  2.07 L
Total loss:  -19.68 L
-------------------
Total Booked + Current PnL: 20.40 L (16.76%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.51 L (58.98%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.97,56.0,H-LC,11.12,171853.0,15676.0,10414.0,-0.80,10.04,6.06,16.71,22.0,1.51,1.23,29.23,X40N,BTT,HEALTHCARE
63,SBIN,863.00,-12.30,59.0,M-LC,4.23,217830.0,16350.0,10870.0,-0.19,8.12,4.99,13.51,60.0,1.50,1.56,21.73,XY25,NTT,BANKS
78,TTKPRESTIG,770.00,104.91,60.0,M-SC,4.71,88307.0,-12470.0,12566.0,0.61,-12.37,14.23,0.09,245.0,-0.99,0.63,16.02,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.02,114650.0,14146.0,16498.0,-0.91,14.08,14.39,30.49,79.0,0.86,0.82,51.04,MH,ATH,METALS
42,ITC,452.00,-37.50,53.0,H-LC,2.91,201810.0,1672.0,19212.0,-0.25,0.84,9.52,10.44,4.0,0.09,1.45,6.45,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-30.16,53.0,H-SC,17.97,199547.0,-407.0,97638.0,1.53,-0.20,48.93,48.63,115.0,-0.00,1.43,11.33,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.42,46.0,H-MC,2.18,136843.0,1055.0,41696.0,0.78,0.78,30.47,31.48,107.0,0.03,0.98,12.99,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.78,248633.0,902.0,74167.0,1.35,0.36,29.83,30.30,92.0,0.01,1.78,13.70,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,60.0,H-MC,5.73,180207.0,399.0,112017.0,-0.15,0.22,62.16,62.52,88.0,0.00,1.29,34.38,XR,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-16.43,39.0,H-LC,5.80,301168.0,-14674.0,127334.0,0.48,-4.65,42.28,35.67,5.0,-0.12,2.16,8.56,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.54,48.0,H-LC,8.42,220634.0,-31134.0,84348.0,-1.54,-12.37,38.23,21.13,27.0,-0.37,1.58,18.68,X40,ATH,PAINTS
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.12,230052.0,-11872.0,75135.0,-0.13,-4.91,32.66,26.15,8.0,-0.16,1.65,8.71,X40,ATH,IT
44,JIOFIN,387.00,-0.26,49.0,H-LC,13.45,270943.0,3578.0,61883.0,1.24,1.34,22.84,24.48,48.0,0.06,1.94,57.05,XY24,BTT,FINANCE
81,UNITDSPR,1644.00,-13.66,49.0,H-LC,8.00,226180.0,-7964.0,56590.0,0.40,-3.40,25.02,20.77,86.0,-0.14,1.62,3.28,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-3.18,60.0,H-LC,1.08,239962.0,-6189.0,85402.0,-0.00,-2.51,35.59,32.18,16.0,-0.07,1.72,32.62,X200,ATH,IT
42,ITC,452.00,-37.50,53.0,H-LC,2.91,201810.0,1672.0,19212.0,-0.25,0.84,9.52,10.44,4.0,0.09,1.45,6.45,X40,NTT,FMCG
51,NESTLEIND,1377.00,-7.84,60.0,H-LC,3.93,283491.0,18065.0,38725.0,-0.50,6.81,13.66,21.40,11.0,0.47,2.03,14.38,XY25,NTT,FMCG
84,VBL,671.64,-16.43,39.0,H-LC,5.80,301168.0,-14674.0,127334.0,0.48,-4.65,42.28,35.67,5.0,-0.12,2.16,8.56,X40N,ATH,FMCG
20,CIPLA,1795.00,-20.12,49.0,H-LC,5.87,212816.0,8316.0,33093.0,-1.35,4.07,15.55,20.25,10.0,0.25,1.53,13.15,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,7969.85,2.53,63.0,H-LC,28.54,164915.0,-21180.0,233586.0,3.01,-11.38,141.64,114.13,15.0,-0.09,1.18,21.45,AR,ATH,ELECTRICAL
76,TITAGARH,1548.00,1.40,71.0,H-SC,4.95,166320.0,-25695.0,104582.0,2.47,-13.38,62.88,41.08,158.0,-0.25,1.19,40.94,XY24,NTT,ENGINEERING
1,ABB,7934.00,-38.41,64.0,H-LC,11.50,257597.0,-4022.0,123234.0,2.25,-1.54,47.84,45.57,7.0,-0.03,1.85,9.61,AR,NTT,ELECTRICAL
82,VAIBHAVGBL,521.00,54.82,62.0,H-SC,9.12,134232.0,-48543.0,159615.0,2.13,-26.56,118.91,60.77,125.0,-0.30,0.96,20.23,XR,NTT,JEWELLERY
65,SHALBY,327.00,1077.03,69.0,M-SC,2.27,160125.0,-21018.0,66484.0,2.01,-11.60,41.52,25.10,235.0,-0.32,1.15,28.32,XY24,NTT,HEALTHCARE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85344.0,-28205.0,68258.0,-1.77,-24.84,79.98,35.27,268.0,-0.41,0.61,108.88,XR,NTT,HOTELS
7,ASIANTILES,137.00,6500.00,40.0,L-SC,5.67,72401.0,-21409.0,98031.0,-1.59,-22.82,135.40,81.67,269.0,-0.22,0.52,40.21,XR,NTT,CERAMICS
6,ASIANPAINT,3465.66,-12.54,48.0,H-LC,8.42,220634.0,-31134.0,84348.0,-1.54,-12.37,38.23,21.13,27.0,-0.37,1.58,18.68,X40,ATH,PAINTS
27,GREENPANEL,537.00,237.21,67.0,M-SC,4.74,150597.0,-32481.0,111457.0,-1.53,-17.74,74.01,43.14,230.0,-0.29,1.08,41.66,XY24,NTT,MISC
56,RELAXO,1176.00,-33.49,52.0,H-SC,1.92,90663.0,-54497.0,130428.0,-1.45,-37.54,143.86,52.31,136.0,-0.42,0.65,21.36,X40N,NTT,FOOTWEAR


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,99.53,60.0,H-SC,12.47,125982.0,-16497.0,36018.0,0.09,-11.58,28.59,13.70,163.0,-0.46,0.90,48.69,OX40N,NTT,BANKS
78,TTKPRESTIG,770.0,104.91,60.0,M-SC,4.71,88307.0,-12470.0,12566.0,0.61,-12.37,14.23,0.09,245.0,-0.99,0.63,16.02,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,58.0,H-SC,1.82,222642.0,-47025.0,83357.0,-0.06,-17.44,37.44,13.47,138.0,-0.56,1.60,12.61,XY24,NTT,PAINTS
19,CERA,9475.0,-20.53,43.0,H-SC,2.17,144888.0,-31015.0,73038.0,-0.36,-17.63,50.41,23.89,149.0,-0.42,1.04,25.13,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-20.52,51.0,H-MC,7.88,107588.0,-24247.0,66468.0,-1.03,-18.39,61.78,32.03,98.0,-0.36,0.77,20.51,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-26.76,51.0,H-LC,13.11,292237.0,-53719.0,128584.0,-0.78,-15.53,44.00,21.64,1.0,-0.42,2.10,3.53,X40,ATH,IT
40,INFY,2275.00,-17.44,53.0,H-LC,9.12,321800.0,8912.0,162766.0,-0.97,2.85,50.58,54.87,3.0,0.05,2.31,9.69,X40,BTT,IT
42,ITC,452.00,-37.50,53.0,H-LC,2.91,201810.0,1672.0,19212.0,-0.25,0.84,9.52,10.44,4.0,0.09,1.45,6.45,X40,NTT,FMCG
84,VBL,671.64,-16.43,39.0,H-LC,5.80,301168.0,-14674.0,127334.0,0.48,-4.65,42.28,35.67,5.0,-0.12,2.16,8.56,X40N,ATH,FMCG
1,ABB,7934.00,-38.41,64.0,H-LC,11.50,257597.0,-4022.0,123234.0,2.25,-1.54,47.84,45.57,7.0,-0.03,1.85,9.61,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,64.0,M-MC,6.94,227407.0,2445.0,165211.0,-0.15,1.09,72.65,74.53,192.0,0.01,1.63,33.69,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.63,54.0,H-MC,3.83,189150.0,24480.0,43486.0,0.35,14.87,22.99,41.28,89.0,0.56,1.36,24.04,X40N,ATH,AC
85,VOLTAS,1918.49,1.58,60.0,H-MC,2.82,213000.0,21258.0,74763.0,1.17,11.09,35.10,50.08,99.0,0.28,1.53,18.64,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.78,248633.0,902.0,74167.0,1.35,0.36,29.83,30.30,92.0,0.01,1.78,13.70,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.48,56.0,H-MC,6.37,210249.0,13221.0,76404.0,0.05,6.71,36.34,45.49,102.0,0.17,1.51,20.63,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.02,114650.0,14146.0,16498.0,-0.91,14.08,14.39,30.49,79.0,0.86,0.82,51.04,MH,ATH,METALS
37,INDIAMART,4850.92,-50.35,55.0,H-SC,12.23,138489.0,15153.0,118616.0,0.47,12.29,85.65,108.45,119.0,0.13,0.99,37.39,AR,ATH,MISC
86,WHIRLPOOL,2270.00,-40.79,51.0,M-SC,1.11,100148.0,8650.0,70104.0,-1.20,9.45,70.00,86.07,223.0,0.12,0.72,45.43,XR,NTT,DURABLES
87,WIPRO,420.00,-11.52,52.0,M-LC,5.70,155533.0,4588.0,104860.0,-0.43,3.04,67.42,72.51,53.0,0.04,1.12,9.25,XR,NTT,IT
36,IEX,219.00,-30.16,53.0,H-SC,17.97,199547.0,-407.0,97638.0,1.53,-0.20,48.93,48.63,115.0,-0.00,1.43,11.33,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85344.0,-28205.0,68258.0,-1.77,-24.84,79.98,35.27,268.0,-0.41,0.61,108.88,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.67,47.0,H-SC,13.68,88761.0,-12202.0,106735.0,1.66,-12.09,120.25,93.63,148.0,-0.11,0.64,32.21,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.54,58.0,H-SC,12.84,95820.0,-2160.0,23763.0,-1.04,-2.20,24.80,22.05,152.0,-0.09,0.69,39.28,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.02,114650.0,14146.0,16498.0,-0.91,14.08,14.39,30.49,79.0,0.86,0.82,51.04,MH,ATH,METALS
8,ATULAUTO,844.00,3818.92,63.0,M-SC,6.75,122118.0,-25250.0,77069.0,-1.38,-17.13,63.11,35.16,236.0,-0.33,0.88,24.73,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3818.92,63.0,M-SC,6.75,122118.0,-25250.0,77069.0,-1.38,-17.13,63.11,35.16,236.0,-0.33,0.88,24.73,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85344.0,-28205.0,68258.0,-1.77,-24.84,79.98,35.27,268.0,-0.41,0.61,108.88,XR,NTT,HOTELS
76,TITAGARH,1548.00,1.40,71.0,H-SC,4.95,166320.0,-25695.0,104582.0,2.47,-13.38,62.88,41.08,158.0,-0.25,1.19,40.94,XY24,NTT,ENGINEERING
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.02,114650.0,14146.0,16498.0,-0.91,14.08,14.39,30.49,79.0,0.86,0.82,51.04,MH,ATH,METALS
61,SAMMAANCAP,326.00,-170.45,62.0,M-SC,2.81,81606.0,-20604.0,113995.0,-0.14,-20.16,139.69,91.37,208.0,-0.18,0.59,32.47,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.51
1,25,42.82
2,50,73.26


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.25
LC    34.01
MC    22.71
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.54
X40      15.03
X40N     12.77
XR       11.31
XY25      9.64
AR        8.52
OX40N     8.04
X200      1.72
MH        1.69
X5K       1.44
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.42
H-LC    25.91
H-MC    19.81
M-SC    13.32
M-LC     7.05
M-MC     2.60
L-SC     1.51
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.15,-14.78,77.36
FMCG,11.66,-4.10,40.52
FINANCE,9.53,-15.90,61.30
BANKS,7.62,-12.64,64.50
MISC,7.38,-11.66,73.76
PAINTS,5.77,-14.83,31.95
ELECTRICAL,5.60,-5.31,63.36
AC,3.59,3.01,29.83
INSURANCE,3.37,-7.14,43.88


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2865289.0,22
XR,1333317.0,14
AR,1306926.0,9
X40,753941.0,10
X40N,630965.0,10
OX40N,525013.0,10
XY25,407313.0,7
SR,245378.0,2
X5K,117202.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3450933.0,29
M-SC,1390041.0,17
H-LC,1261829.0,15
H-MC,1189101.0,15
M-LC,395136.0,5
M-MC,314720.0,2
L-SC,250442.0,3
L-MC,60315.0,1
L-LC,38111.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1035984.0      6
M-SC       XY24       801348.0      7
H-SC       AR         787819.0      5
           XR         772700.0      7
H-MC       XY24       489150.0      4
H-LC       X40        470045.0      5
           AR         356820.0      2
M-MC       XY24       314720.0      2
H-SC       X40N       296755.0      4
           OX40N      248913.0      4
           SR         245378.0      2
H-LC       X40N       227431.0      4
H-MC       X40        214100.0      4
           XY25       184089.0      2
L-SC       XR         166289.0      2
M-SC       AR         162287.0      2
M-LC       XY24       162204.0      2
M-SC       OX40N      125479.0      4
M-LC       X5K        117202.0      1
M-SC       XR         117136.0      2
           XY25       113995.0      1
H-MC       XR         112017.0      1
           X40N       106779.0      2
M-LC       XR         104860.0      1
H-LC       X200        85402.0      1
L-SC       OX40N       84153.0      1
M-SC       X40         69796.0      1
H-MC       OX40N       66468.0      1
H-SC       MH          63384.0      1
H-LC       XY24        61883.0      1
L-MC       XR          60315.0      1
H-LC       XY25        60248.0      2
L-LC       XY25        38111.0      1
H-MC       MH          16498.0      1
M-LC       XY25        10870.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
